In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow_datasets as tfds


In [ ]:
import google.protobuf

print("TF:", tf.__version__)
print("TFDS:", tfds.__version__)
print("protobuf:", google.protobuf.__version__)


In [ ]:
dataset, info = tfds.load("tf_flowers", with_info=True, as_supervised=True)
train_ds = dataset["train"]
print(info)


In [ ]:
IMG_SIZE = 224   # standard input size for many pretrained models
BATCH_SIZE = 32  # how many images to feed the model at once

# Function to resize and normalize each image
def format_example(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))  # resize to 224x224
    image = image / 255.0  # normalize pixel values [0,255] -> [0,1]
    return image, label

# Apply preprocessing to the dataset
train_ds = train_ds.map(format_example)

# Shuffle and batch the data
train_ds = train_ds.shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


In [ ]:
# Take one batch from the dataset
for images, labels in train_ds.take(1):
    plt.figure(figsize=(10, 10))
    
    for i in range(9):  # show 9 images
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy())  # convert Tensor to NumPy for plotting
        plt.title(int(labels[i].numpy()))  # show class index as title
        plt.axis("off")
